# Import all the required libraries

In [24]:
import numpy as np
import pandas as pd
import pickle
from sklearn import preprocessing
from sklearn import model_selection
import seaborn as sns
from collections import Counter
import imblearn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_error, confusion_matrix
from imblearn.under_sampling import NearMiss
from category_encoders import LeaveOneOutEncoder
import warnings
warnings.filterwarnings("ignore")

## Import Dataset

In [2]:
customer_df = pd.read_csv('../input/brazilian-ecommerce/olist_customers_dataset.csv',header= 0,encoding='unicode_escape')
order_df = pd.read_csv('../input/brazilian-ecommerce/olist_orders_dataset.csv',header= 0,encoding='unicode_escape')
order_items_df = pd.read_csv('../input/brazilian-ecommerce/olist_order_items_dataset.csv',header= 0,encoding='unicode_escape')
products_df = pd.read_csv('../input/brazilian-ecommerce/olist_products_dataset.csv',header= 0,encoding='unicode_escape')
sellers_df = pd.read_csv('../input/brazilian-ecommerce/olist_sellers_dataset.csv',header= 0,encoding='unicode_escape')
order_payments_df = pd.read_csv('../input/brazilian-ecommerce/olist_order_payments_dataset.csv',header= 0,encoding='unicode_escape')

geolocation_df = pd.read_csv('../input/brazilian-ecommerce/olist_geolocation_dataset.csv',header= 0,encoding='unicode_escape')
pd.set_option('display.max_columns',None)


In [ ]:
order_items_df.shape

## Join datasets
#### inner join

In [3]:
olist_df = pd.merge(customer_df,                     # Inner join
                      order_df,
                      on = "customer_id",
                      how = "inner")
olist_df = pd.merge(olist_df,                     # Inner join
                      order_items_df,
                      on = "order_id",
                      how = "inner")

olist_df = pd.merge(olist_df,                     # Inner join
                      products_df,
                      on = "product_id",
                      how = "inner")

olist_df = pd.merge(olist_df,                     # Inner join
                      sellers_df,
                      on = "seller_id",
                      how = "inner")

olist_df = pd.merge(olist_df,                     # Inner join
                      order_payments_df,
                      on = "order_id",
                      how = "inner")

# olist_df = pd.merge(left=olist_df,right=geolocation_df, left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix')

olist_df.head()

In [ ]:
# def basic_info(df):
#     print('Total data-points :',df.size)
#     print('Total features: ',df.shape[1])
#     numerical_features = [f for f in df.columns if df[f].dtypes!='O']
#     print('Total numerical features :',len(numerical_features))
#     cat_features = [c for c in df.columns if df[c].dtypes=='O']
#     print('Total categorical features :',len(cat_features))
#     print('Total rows: ',df.shape[0])
#     print("\nFeature lists containing null values in percentage:\n")
#     for column in df.columns:
#         if df[column].isnull().sum() > 0:
#             print(column, ': {:.4%}'.format(df[column].isnull().sum() / df[column].shape[0]))
# basic_info(olist_df)


In [ ]:
def heatMap(df_heatmap):
    ftr_corr = df_heatmap.corr()
    top_corr=ftr_corr.index
    plot.figure(figsize=(20,15))
    g=sns.heatmap(df_heatmap[top_corr].corr(),annot=True)
    df_heatmap.columns.tolist()
    
# heatMap(olist_df)
# olist_df.shape

In [ ]:
order_df.head()

In [4]:
def pieChart(feature_df,feature_title):
    order_status_ = feature_df.value_counts()
    order_status_label_ = order_status_.keys()
    plt.figure(figsize=(10,10))
    plt.pie(order_status_,labels=order_status_label_,autopct='%1.2f%%')
    plt.legend(title=feature_title)
    plt.show()
    


pieChart(olist_df['order_status'],"Order Status")

order_df.shape

#### Order_df features explained
* order_estimated_delivery_date = Shows the estimated delivery date that was informed to customer at the purchase moment.
* order_delivered_customer_date = Shows the actual order delivery date to the customer.
* order_delivered_carrier_date = Shows the order posting timestamp. When it was handled to the logistic partner.
* shipping_limit_date = Shows the seller shipping limit date for handling the order over to the logistic partner.  ##
* order_approved_at = Shows the payment approval timestamp.
* order_purchase_timestamp = Shows the purchase timestamp.


In [5]:
def getCtgCols(df):
    cols = df.columns.tolist()
    num_cols = df._get_numeric_data().columns
    return list(set(cols) - set(num_cols))

def getNumCols(df):
    return df._get_numeric_data().columns

In [6]:
# olist_df = olist_df[olist_df['order_status'] == 'delivered']
# olist_df.drop(['order_status'],axis=1,inplace=True)
olist_df.head()


In [7]:


olist_df['order_purchase_timestamp'] = pd.to_datetime(olist_df['order_purchase_timestamp'], format="%Y-%m-%d %H:%M:%S")
olist_df['order_approved_at'] = pd.to_datetime(olist_df['order_approved_at'], format="%Y-%m-%d %H:%M:%S")
olist_df['order_delivered_carrier_date']  = pd.to_datetime(olist_df['order_delivered_carrier_date'], format="%Y-%m-%d %H:%M:%S")
olist_df['order_delivered_customer_date'] = pd.to_datetime(olist_df['order_delivered_customer_date'], format="%Y-%m-%d %H:%M:%S")
olist_df['order_estimated_delivery_date'] = pd.to_datetime(olist_df['order_estimated_delivery_date'], format="%Y-%m-%d")
olist_df['shipping_limit_date'] = pd.to_datetime(olist_df['shipping_limit_date'], format="%Y-%m-%d %H:%M:%S")

order_df.dtypes

In [8]:
olist_df['purchase_year'] = olist_df['order_purchase_timestamp'].dt.year
olist_df['purchase_month'] = olist_df['order_purchase_timestamp'].dt.month
olist_df['purchase_day'] = olist_df['order_purchase_timestamp'].dt.day
olist_df['purchase_hour'] = olist_df['order_purchase_timestamp'].dt.hour
olist_df['purchase_minute'] = olist_df['order_purchase_timestamp'].dt.minute

olist_df['approved_year'] = olist_df['order_approved_at'].dt.year
olist_df['approved_month'] = olist_df['order_approved_at'].dt.month
olist_df['approved_day'] = olist_df['order_approved_at'].dt.day
olist_df['approved_hour'] = olist_df['order_approved_at'].dt.hour
olist_df['approved_minute'] = olist_df['order_approved_at'].dt.minute

olist_df['delivered_carrier_year'] = olist_df['order_delivered_carrier_date'].dt.year
olist_df['delivered_carrier_month'] = olist_df['order_delivered_carrier_date'].dt.month
olist_df['delivered_carrier_day'] = olist_df['order_delivered_carrier_date'].dt.day
olist_df['delivered_carrier_hour'] = olist_df['order_delivered_carrier_date'].dt.hour
olist_df['delivered_carrier_minute'] = olist_df['order_delivered_carrier_date'].dt.minute

olist_df['delivered_customer_year'] = olist_df['order_delivered_customer_date'].dt.year
olist_df['delivered_customer_month'] = olist_df['order_delivered_customer_date'].dt.month
olist_df['delivered_customer_day'] = olist_df['order_delivered_customer_date'].dt.day
olist_df['delivered_customer_hour'] = olist_df['order_delivered_customer_date'].dt.hour
olist_df['delivered_customer_minute'] = olist_df['order_delivered_customer_date'].dt.minute

olist_df['estimated_delivery_year'] = olist_df['order_estimated_delivery_date'].dt.year
olist_df['estimated_delivery_month'] = olist_df['order_estimated_delivery_date'].dt.month


olist_df['shipping_limit_year'] = olist_df['shipping_limit_date'].dt.year
olist_df['shipping_limit_month'] = olist_df['shipping_limit_date'].dt.month
olist_df['shipping_limit_day'] = olist_df['shipping_limit_date'].dt.day
olist_df['shipping_limit_hour'] = olist_df['shipping_limit_date'].dt.hour
olist_df['shipping_limit_minute'] = olist_df['shipping_limit_date'].dt.minute

In [9]:
# order_purchase_timestamp = olist_df['order_purchase_timestamp'].astype("int64")
olist_df['late_delivery'] = (olist_df['order_estimated_delivery_date'] - olist_df['order_delivered_customer_date']).dt.days

olist_df['late_delivery'] =   np.where(olist_df['late_delivery'] < 0,1,0)

pieChart(olist_df['late_delivery'],"late_delivery")
# olist_df.head(5)

#### Check for null values

In [37]:
is_null_exist_ = olist_df.isnull().values.sum()
olist_df.dropna(axis=0,inplace=True)
print(is_null_exist_)
is_null_exist_ = olist_df.isnull().values.sum()
print(is_null_exist_)

## Pre Modeling

In [39]:
olist_df.columns.tolist()
model_df = olist_df[['purchase_year',
'purchase_month',
'purchase_day',
'purchase_hour',
'purchase_minute',
 'approved_year',
  'approved_month',
  'approved_day',
'approved_hour',
  'approved_minute',                   
  'delivered_carrier_year',
 'delivered_carrier_month',
  'delivered_carrier_day',
 'estimated_delivery_year',
'delivered_carrier_hour',
 'delivered_carrier_minute',
'estimated_delivery_month',
'shipping_limit_year',
'shipping_limit_month',
'shipping_limit_day',
'shipping_limit_hour',
'shipping_limit_minute',                   
'customer_zip_code_prefix',
 'customer_city',
 'customer_state', 
 'price',
 'freight_value',
 'product_category_name',
'seller_zip_code_prefix',
 'seller_city',
 'seller_state',
 'payment_sequential',
 'payment_type',
 'payment_installments',
 'payment_value']].copy()
model_df.isnull().values.sum()



Y = olist_df['late_delivery'].copy()
Counter(Y)

In [41]:
model_df_shape = model_df.shape
y_len = len(Y)

print(model_df_shape,y_len)

### LeaveOneOutEncoder && Undersampling

In [42]:
model_df_ctg_ftr = getCtgCols(model_df)

encoder_ = LeaveOneOutEncoder(model_df_ctg_ftr)
model_df = encoder_.fit_transform(model_df,Y)

## Undersampling

In [43]:
# Undersample imbalanced dataset with NearMiss-1
undersample = NearMiss(version=1)
X,y = undersample.fit_resample(model_df,Y)

In [45]:
model_df_shape = X.shape
y_len = len(y)

print(model_df_shape,y_len)
Counter(y)

### Split The Dataset for Training && Testing Purpose.

In [47]:
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=.3,random_state=0)

## Data Modeling 
### Random Forest Classifier

In [48]:
model_rfc = RandomForestClassifier(random_state=0)

model_rfc.fit(x_train,y_train)
y_pred = model_rfc.predict(x_test)
score = r2_score(y_test,y_pred)
print(score)

In [51]:
ftr_to_use = ['purchase_year',
'purchase_month',
'purchase_day',
'purchase_hour',
'purchase_minute',
 'approved_year',
  'approved_month',
  'approved_day',
'approved_hour',
  'approved_minute',                   
  'delivered_carrier_year',
 'delivered_carrier_month',
  'delivered_carrier_day',
 'estimated_delivery_year',
'delivered_carrier_hour',
 'delivered_carrier_minute',
'estimated_delivery_month',
'shipping_limit_year',
'shipping_limit_month',
'shipping_limit_day',
'shipping_limit_hour',
'shipping_limit_minute',                   
'customer_zip_code_prefix',
 'customer_city',
 'customer_state', 
 'price',
 'freight_value',
 'product_category_name',
'seller_zip_code_prefix',
 'seller_city',
 'seller_state',
 'payment_sequential',
 'payment_type',
 'payment_installments',
 'payment_value']

feature_importances = pd.DataFrame(model_rfc.feature_importances_,
index = ftr_to_use,
columns=["importance"]).sort_values("importance",
ascending=False)
num = min([50,len(ftr_to_use)])
ylocs = np.arange(num)
# get the feature importance for top num and sort in reverse order
values_to_plot = feature_importances.iloc[:num].values.ravel()[::-1]
feature_labels = list(feature_importances.iloc[:num].index)[::-1]
plt.figure(num=None, figsize=(6, 6), dpi=80, facecolor="w", edgecolor="k");
plt.barh(ylocs, values_to_plot, align = "center")
plt.ylabel("Features")
plt.xlabel("Importance Score")
plt.title("Feature Importance Score — Random Forest")
plt.yticks(ylocs, feature_labels)
plt.show()